In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

In [2]:
hydroswot = pd.read_csv('dataset/SWOT_ADCP_Dataset.csv', converters={'site_no': str}, low_memory=False)
hydroswot.shape

(223022, 75)

In [5]:
candidate_columns=['site_no', 'site_tp_cd', 'q_va', 'stream_wdth_va', 'xsec_area_va', 'mean_depth_va', 'max_depth_va']
df = hydroswot[candidate_columns]
df = df.dropna(axis=0, how='any', subset=candidate_columns)
print(df.shape)

(55240, 7)


In [6]:
df = df[(df != 0).all(axis=1)]
print(df.shape)

(55231, 7)


In [7]:
hydro_pos_q = df[df.groupby('site_no')['q_va'].transform(lambda x: (x<0).sum()) == 0]
print(hydro_pos_q.shape)

(49022, 7)


In [8]:
hydro_pos_q = hydro_pos_q[hydro_pos_q['site_tp_cd'].isin(['ST'])]
print(hydro_pos_q.shape)

(47522, 7)


In [9]:
hydro_pos_q.reset_index(drop=True, inplace=True)

print("site locations with positive discharge: ")
hydro_pos_q['site_no'].value_counts().describe()

site locations with positive discharge: 


count    5914.000000
mean        8.035509
std        10.050780
min         1.000000
25%         2.000000
50%         4.000000
75%        11.000000
max       159.000000
Name: count, dtype: float64

In [10]:
hydro_pos_q.describe()

,q_va,stream_wdth_va,xsec_area_va,mean_depth_va,max_depth_va
count,4.752200e+04,47522.000000,47522.000000,47522.000000,47522.000000
mean,1.100066e+04,239.193869,3431.116848,6.682162,9.857616
std,5.903527e+04,397.990026,14575.380645,7.269928,10.752993
min,5.300000e-02,2.550000,1.080000,0.520000,0.680000
25%,2.391915e+02,69.230000,195.000250,2.677000,4.085000
50%,8.000850e+02,122.638000,529.430000,4.460000,6.791000
75%,3.128004e+03,244.187250,1585.674250,7.842750,11.407000
max,2.257500e+06,16361.950000,357000.000000,118.770000,205.670000
